## Cluster generating

In [21]:
import pandas as pd
import numpy as np

In [3]:
train_ds = pd.read_csv('https://raw.githubusercontent.com/vladseve7n/ITMO.DUBL/main/dataset/train.csv').drop('pair_id', axis=1)

In [4]:
train_ds

,name_1,name_2,is_duplicate
0,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
1,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
2,"Rishichem Distributors Pvt., Ltd.",Dsa,0
3,Powermax Rubber Factory,Co. One,0
4,Tress A/S,Longyou Industries Park Zhejiang,0
...,...,...,...
497814,BIT-MAT PRODUCTS,The Goodyear Tire and Rubber Company,0
497815,"Bnd Trading Co., Ltd.",Zhong Shan Yue Liang Economy& Trade Imp. & Exp...,0
497816,"Xeikon Industrial Co., Ltd. Of Dongguan City","Yi Cheng Trading Co., Ltd. Of Dongguan City",0
497817,"Shanghai Kechuan Trading Co., Ltd.",Shanghai M&G Stationery Inc.,0


Разделим датасет по кластерам

In [52]:
clusters = []

In [53]:
unique_names = list(np.unique(np.append(train_ds.name_1.values, train_ds.name_2.values)))
print(f'Всего уникальных имен {len(unique_names)}')

Всего уникальных имен 18022


In [54]:
ds = train_ds.copy()

In [ ]:
for name in unique_names:
    # Для каждого уникального имени ищем пары
    par_of_names = ds[((ds.name_1 == name) | (ds.name_2 == name)) & (ds.is_duplicate == 1)]
    unique_values = list(np.unique(np.append(par_of_names.name_1.values, par_of_names.name_2.values)))
    print(unique_values)
    # Если пары существуют
    if len(unique_values) > 0:
        # Идем глубже и для каждого значеня в паре ищем еще пары
        unique_values.remove(name)
        depper_unique_values = []

        for val in unique_values:
            par_of_names = ds[((ds.name_1 == val) | (ds.name_2 == val)) & (ds.is_duplicate == 1)]
            depper_unique_values.extend(list(np.unique(np.append(par_of_names.name_1.values, par_of_names.name_2.values))))

        unique_values.extend(depper_unique_values)
        unique_values.append(name)
        unique_values = list(set(unique_values))
        print(unique_values)
        # Добавляем кластер в переменную 
        clusters.append(unique_values)
        # Удаляем значения из датасета, если мы его уже проверили и отнесили к кластеру
        for val in unique_values:
            unique_names.remove(val)
            ds.drop(ds[ds.name_1 == val].index, inplace = True)
            ds.drop(ds[ds.name_2 == val].index, inplace = True)
            print(len(unique_names), len(ds))
    else:
        unique_names.remove(name)
        ds.drop(ds[ds.name_1 == name].index, inplace = True)
        ds.drop(ds[ds.name_2 == name].index, inplace = True)
        print(len(unique_names), len(ds))

In [ ]:
clusters

In [71]:
train_ds[((train_ds['name_1'] == 'BASF CENRTAL ASIA LLP') | (train_ds['name_2'] == 'BASF CENRTAL ASIA LLP')) & (train_ds['is_duplicate'] == 1)]

,name_1,name_2,is_duplicate
378355,BASF CENRTAL ASIA LLP,BASF Central Asia LLP,1


In [75]:
train_ds[((train_ds['name_1'] == 'BASF Central Asia LLP') | (train_ds['name_2'] == 'BASF Central Asia LLP')) & (train_ds['is_duplicate'] == 1)]

,name_1,name_2,is_duplicate
378355,BASF CENRTAL ASIA LLP,BASF Central Asia LLP,1


Как мы видим в кластерах есть проблема. Это очевидно, что компания BASF имеет больше элементов в своем кластере, но такие элементы, как: 'BASF Central Asia LLP', 'BASF CENRTAL ASIA LLP' из него выпали, что говорит нам о том, что не все кластеры в данном датасеты могу быть восстановленны

In [72]:
import json
with open('clusters.json', 'w') as source:
    json.dump({'clusters': clusters}, source)

Добавим в cluster'ы название уникальных компаний без пары в виде уникального кластера

In [76]:
unique_names = list(np.unique(np.append(train_ds.name_1.values, train_ds.name_2.values)))
print(f'Всего уникальных имен {len(unique_names)}')

Всего уникальных имен 18022


In [85]:
unique_values = []

In [86]:
for name in unique_names:
    for clus in clusters:
        if name in clus:
            continue
    unique_values.append(name)

In [ ]:
unique_values

In [90]:
for val in unique_values:
    clusters.append([val])

In [91]:
with open('clusters_with_unique.json', 'w') as source:
    json.dump({'clusters': clusters}, source)